# Mini-Projeto 1 - Analisando Dados do Uber com Spark 

Dataset: https://github.com/fivethirtyeight/uber-tlc-foil-response

Esse conjunto de dados contém dados de mais de 4,5 milhões de captações Uber na cidade de Nova York de abril a setembro de 2014 e 14,3 milhões de captações Uber de janeiro a junho de 2015. Dados em nível de viagem sobre 10 outras empresas de veículos de aluguel (FHV) bem como dados agregados para 329 empresas de FHV, também estão incluídos. Todos os arquivos foram recebidos em 3 de agosto, 15 de setembro e 22 de setembro de 2015.

1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

2- Qual o total de veículos que passaram pela base B02617?

3- Qual o total de corridas por base? Apresente de forma decrescente.

In [1]:
from pandas import read_csv

In [2]:
uber = read_csv('7-Arquivos-Cap07/data/uber.csv')

In [3]:
type(uber)

pandas.core.frame.DataFrame

In [4]:
# convertendo para um datafrema spark
uber_rdd = sqlContext.createDataFrame(uber)

In [5]:
type(uber_rdd)

pyspark.sql.dataframe.DataFrame

In [6]:
uber_rdd.count()

354

In [7]:
# criando rdd a partir de um arquivo csv
uber_rdd = sc.textFile('7-Arquivos-Cap07/data/uber.csv')

In [8]:
type(uber_rdd)

pyspark.rdd.RDD

In [11]:
#. total de registros
uber_rdd.count()

355

In [13]:
# primeiro registro
uber_rdd.first()

'dispatching_base_number,date,active_vehicles,trips'

In [14]:
# dividindo o arquivo em colunas separadas por , 
uber_linhas = uber_rdd.map(lambda x: x.split(','))

In [16]:
type(uber_linhas)

pyspark.rdd.PipelinedRDD

In [19]:
#numero de base de carros do uber
uber_linhas.map(lambda linha: linha[0]).distinct().count() -1 

6

In [21]:
# base de carros do uber
uber_linhas.map(lambda linha: linha[0]).distinct().collect()

['dispatching_base_number',
 'B02765',
 'B02682',
 'B02598',
 'B02512',
 'B02764',
 'B02617']

In [25]:
# total de veículos que passaram pela base B02617
uber_linhas.filter(lambda linha: 'B02617' in linha).count()

59

In [31]:
# Total de dias em que o número de corridas foi superior a 16.000

uber_linhas.filter(lambda linha: "B02617" in linha).filter(lambda linha: int(linha[3]) > 16000).count()

4

In [32]:
uber_linhas.filter(lambda linha: "B02617" in linha).filter(lambda linha: int(linha[3]) > 16000).collect()

[['B02617', '2/13/2015', '1590', '16996'],
 ['B02617', '2/14/2015', '1486', '16999'],
 ['B02617', '2/20/2015', '1574', '16856'],
 ['B02617', '2/21/2015', '1443', '16098']]

In [38]:
uberRDD2 = sc.textFile("7-Arquivos-Cap07/data/uber.csv").filter(lambda line: "base" not in line).map(lambda line:line.split(","))

In [39]:
#aplicando redução para calcular total por base
uberRDD2.map(lambda pk: (pk[0], int(pk[3]))).reduceByKey(lambda k,v: k+v).collect()

[('B02765', 193670),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02512', 93786),
 ('B02764', 1914449),
 ('B02617', 725025)]

In [41]:
#aplicando redução para calcular total por base em ordem descrecente

uberRDD2.map(lambda pk: (pk[0], int(pk[3]))).reduceByKey(lambda k,v: k+v).sortBy(keyfunc=(lambda x: x[1])).collect()

[('B02512', 93786),
 ('B02765', 193670),
 ('B02598', 540791),
 ('B02682', 662509),
 ('B02617', 725025),
 ('B02764', 1914449)]